### Quest
- 목표변수:SICK_SYM1, 설명변수 5개 사용
- DecisionTreeClassifier 전처리, 학습, 평가
- option : SICK_SYM1 항목은 8개만 선택

In [1]:
import pandas as pd
import re

In [2]:
df_JCNBM = pd.read_csv('../../../datasets/NSC_BND_M20.csv')

C:\Users\01-10\AppData\Local\Temp\ipykernel_12564\1035179799.py:1: DtypeWarning: Columns (3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_JCNBM = pd.read_csv('../../../datasets/NSC_BND_M20.csv')


In [3]:
df_JCNBM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137163 entries, 0 to 137162
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          137163 non-null  int64  
 1   _id                 137163 non-null  object 
 2   RN_INDI             137163 non-null  int64  
 3   BTH_YYYY            137163 non-null  object 
 4   DTH_YYYYMM          10269 non-null   float64
 5   COD1                8473 non-null    object 
 6   COD2                998 non-null     object 
 7   RN_KEY              137163 non-null  int64  
 8   RN_INST             137163 non-null  int64  
 9   MDCARE_STRT_DT      137163 non-null  int64  
 10  FORM_CD             137163 non-null  int64  
 11  MCARE_SUBJ_CD       137163 non-null  int64  
 12  SICK_SYM1           137163 non-null  object 
 13  OFIJ_TYPE           134428 non-null  object 
 14  OPRTN_YN            137163 non-null  int64  
 15  MDCARE_DD_CNT       137163 non-nul

In [4]:
df_JCNBM.columns

Index(['Unnamed: 0', '_id', 'RN_INDI', 'BTH_YYYY', 'DTH_YYYYMM', 'COD1',
       'COD2', 'RN_KEY', 'RN_INST', 'MDCARE_STRT_DT', 'FORM_CD',
       'MCARE_SUBJ_CD', 'SICK_SYM1', 'OFIJ_TYPE', 'OPRTN_YN', 'MDCARE_DD_CNT',
       'VSHSP_DD_CNT', 'TOT_PRSC_DD_CNT', 'MCARE_RSLT_TYPE', 'EDC_ADD_RT',
       'ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT', 'STD_YYYY'],
      dtype='object')

In [5]:
df_JCNBM = df_JCNBM[['SICK_SYM1','TOT_PRSC_DD_CNT', 'MCARE_RSLT_TYPE', 'EDC_ADD_RT','ED_RC_TOT_AMT', 'EDC_SBA']]
df_JCNBM['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [6]:
df_JCNBM.isnull().sum()

SICK_SYM1             0
TOT_PRSC_DD_CNT       0
MCARE_RSLT_TYPE    3205
EDC_ADD_RT            0
ED_RC_TOT_AMT         0
EDC_SBA               0
dtype: int64

In [7]:
df_JCNBM = df_JCNBM.dropna()

In [12]:
def convert_to_numeric(value):
    if re.match(r'[A-Z]', value):
        return re.sub(r'[A-Z]', lambda x: str(ord(x.group()) - ord('A')), value)
    elif '_' in value:
        return re.sub('_', '1', value)
    else:
        return value
df_JCNBM['SICK_SYM1'] = df_JCNBM['SICK_SYM1'].apply(convert_to_numeric)
df_JCNBM

,SICK_SYM1,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA
0,11028,3,5.0,0.15,10590,3000
1,9030,2,5.0,0.15,16520,4950
2,9030,3,5.0,0.15,27620,9000
3,10291,3,5.0,0.15,12920,3000
4,12545,0,5.0,0.15,14230,3000
...,...,...,...,...,...,...
137158,9209,0,1.0,0.15,11280,2300
137159,9209,0,1.0,0.15,19640,3700
137160,9209,0,1.0,0.15,13470,2800
137161,9209,0,1.0,0.15,13530,2600


In [13]:
df_JCNBM['SICK_SYM1'].value_counts()


9209     8221
810      4205
131      3161
51       3089
8109     2913
         ... 
889         1
626         1
2799        1
10658       1
15289       1
Name: SICK_SYM1, Length: 3339, dtype: int64

In [14]:
top_8_values = df_JCNBM['SICK_SYM1'].value_counts().head(8).index.tolist()
top_8_values

['9209', '810', '131', '51', '8109', '9039', '9029', '900']

In [ ]:
top_8_indices = df_JCNBM[df_JCNBM['SICK_SYM1'].isin(top_8_values)]
top_8_indices

#### 정형화

In [20]:
target_train = df_JCNBM['SICK_SYM1'] # 범주가 여러개인 목표변수 
features_train = df_JCNBM.drop(columns=['SICK_SYM1']) # 목표변수 뺀 나머지 설명변수

In [21]:
target_train.shape, features_train.shape

((133958,), (133958, 5))

#### learning 알고리즘 = 모델 학습
- 다항분류면 Tree 형식으로 진행됨. 
    - from sklearn.tree import DecisionTreeClassifier
    - from sklearn.tree import DecisionTreeRegressor # Regression도 Tree구조로 가능함. 

In [22]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(features_train, target_train)

DecisionTreeClassifier()

In [24]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [25]:
features_train.columns.to_list()

['TOT_PRSC_DD_CNT',
 'MCARE_RSLT_TYPE',
 'EDC_ADD_RT',
 'ED_RC_TOT_AMT',
 'EDC_SBA']

In [26]:
%matplotlib tk

In [ ]:
# plot_tree(model, feature_names=features_train.columns.to_list()) ##plot_tree(모델,컬럼,목표변수)
plt.figure(figsize=(40,25))
plot_tree(model, feature_names=features_train.columns.to_list(), filled=True)
plt.show()

#### 평가
- 예측한 값이 얼마가 일치하는지 평가를 한다. 

In [ ]:
target_predict = model.predict(features_train)
target_predict.shape

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(target_train, target_predict)